# Interpreters

> infix to postix, interpreting formulas


## Infix to postfix

Iterables produce Iterators via `iter()` such as all Collections.
Iterator yield the next element via `next()`. By convention, all Iterators
are also Iterables; the standard implementation being to return self.
Some examples:

* `(0, 1, 2, 3)` is a tuple, no iterator
* `range(4)` is a tuple, no iterator
* `naturals()` is an iterator
* `(1 / k for k in naturals())` is an iterator

## A simple grammar

todo

##  Compiling FORTH

todo


